In [216]:
import csv
import numpy as np
from sklearn.metrics import log_loss

In [183]:
year = 2005
with open('data-'+ str(year) +'/game.csv', 'r') as f:
    reader = csv.reader(f)
    Game = list(reader)
Game = Game[1:]

with open('data-' + str(year) + '/game-statistics.csv', 'r') as f:
    reader = csv.reader(f)
    GameStatistics = list(reader)
    GameStatistics = GameStatistics[1:]
    
with open('data-' + str(year) + '/team-game-statistics.csv', 'r') as f:
    reader = csv.reader(f)
    TeamGameStatistics = list(reader)
    TeamGameStatistics = TeamGameStatistics[1:]

    
#with open('data-2005/team-game-statistics.csv', 'r') as f:
 # reader = csv.reader(f)
  #TeamGameStatistics = list(reader)

In [184]:
print(len(Game))

718


In [185]:
print(Game[0])
#TeamGameStatistics[100]
print(len(TeamGameStatistics))

['0086016420050901', '09/01/2005', '86', '164', '25', 'TEAM']
1436


In [186]:
TeamGameStatistics[0]

['5',
 '0005055920050910',
 '21',
 '23',
 '0',
 '46',
 '26',
 '362',
 '3',
 '0',
 '0',
 '5',
 '144',
 '0',
 '2',
 '-2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '3',
 '3',
 '0',
 '0',
 '0',
 '0',
 '0',
 '24',
 '7',
 '260',
 '5',
 '228',
 '0',
 '0',
 '0',
 '4',
 '2',
 '59',
 '18',
 '2.0',
 '4',
 '0.0',
 '0',
 '0',
 '1',
 '2',
 '0',
 '4',
 '12',
 '1',
 '1433',
 '5',
 '42',
 '13',
 '2',
 '3',
 '0',
 '2',
 '1',
 '1']

In [187]:
games = {}
games.update({'gameID':0})
games.update({'date':0})
games.update({'attendance':0})
games.update({'duration':0})

In [188]:
games

{'attendance': 0, 'date': 0, 'duration': 0, 'gameID': 0}

In [189]:
#Assuming no anomolies in game data in TEAMGAMESTATS correspnoding to games in GAME
#Add fields from TEAMSGAMESTATS in this function
def SearchGame(GameID, TeamGameStatistics, HomeTeam, VisitTeam):
    count = 0
    for i,gamestats in enumerate(TeamGameStatistics):
        if(gamestats[1] == GameID):
            if(gamestats[0] == HomeTeam):
                HomeTeamStats = gamestats
            else:
                VisitTeamStats = gamestats
            count = count + 1
        if(count == 2):
            return(HomeTeamStats,VisitTeamStats)
    print("Loop ended without returning! :( ")

In [190]:
def listOfDictionaries(GameList, Game, GameStatistics, TeamGameStatistics):
    newGame = {}
    newGame['Date'] = Game[1]
    newGame['GameID'] = Game[0]
    newGame['Attendance'] = GameStatistics[1]
    newGame['Duration'] = GameStatistics[2]
    newGame['HomeTeam'] = int(Game[3])
    newGame['VisitTeam'] = int(Game[2])
    HomeTeamStats, VisitTeamStats = SearchGame(Game[0],TeamGameStatistics,Game[3],Game[2])
    newGame['HT-Points'] = int(HomeTeamStats[35])
    newGame['VT-Points'] = int(VisitTeamStats[35])
    GameList.append(newGame)
    return GameList

In [191]:
GameList = [] #List of dictionaries, based on sorted order of games
TeamList = [] #List of Teams in sorted order of Team codes
count = 0
for i in range(len(Game)):
    GameList = listOfDictionaries(GameList, Game[i], GameStatistics[i], TeamGameStatistics)
    GameList
    if(GameList[i]['HomeTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['HomeTeam'])
    if(GameList[i]['VisitTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['VisitTeam'])
        
TeamList.sort()

In [192]:
GameList[0]

{'Attendance': '40000',
 'Date': '09/01/2005',
 'Duration': '164',
 'GameID': '0086016420050901',
 'HT-Points': 38,
 'HomeTeam': 164,
 'VT-Points': 0,
 'VisitTeam': 86}

In [193]:
NumberOfWins = []
NumberOfMatches = []
for i,team in enumerate(TeamList):
    NumberOfWins.append(0)
    NumberOfMatches.append(0)

In [194]:
len(TeamList)

166

In [195]:
#win ratios of home team, win ratio of visit team
X_train = []
X_test = []
#1 if home team wins, zero otherwise
Y_train = []
Y_test = []

In [196]:
#Skip first 100 games
gamecount = 0
for i,game in enumerate(GameList[:100]):
    gamecount = gamecount + 1
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    if(game['HT-Points'] >= game['VT-Points']):
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1        
    

In [197]:
gamecount

100

In [198]:
for i,team in enumerate(TeamList):
    if(NumberOfMatches[i] < NumberOfWins[i]):
        print("Matches: ", NumberOfMatches[i], " Wins: ", NumberOfWins[i])

In [199]:
for i,game in enumerate(GameList[100:]):
    temp = []
    gamecount = gamecount + 1
    print("For ",i,": ")
    #print(game)
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    print(NumberOfWins[htindex])
    print(NumberOfMatches[htindex])
    print(NumberOfWins[vtindex])
    print(NumberOfMatches[vtindex])
    if(NumberOfMatches[htindex] > 0):
        temp.append(NumberOfWins[htindex]/NumberOfMatches[htindex])
    else:
        temp.append(0.0)
    if(NumberOfMatches[vtindex] > 0):
        temp.append(NumberOfWins[vtindex]/NumberOfMatches[vtindex])
    else:
        temp.append(0.0)
    X_train.append(temp)
    if(game['HT-Points'] >= game['VT-Points']):
        Y_train.append(1)
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        Y_train.append(0)
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1

For  0 : 
1
1
1
1
For  1 : 
0
1
0
0
For  2 : 
1
1
0
0
For  3 : 
0
1
0
1
For  4 : 
0
1
0
0
For  5 : 
1
1
0
1
For  6 : 
0
1
0
0
For  7 : 
0
1
1
1
For  8 : 
1
1
1
1
For  9 : 
0
1
0
1
For  10 : 
1
1
0
0
For  11 : 
1
1
0
0
For  12 : 
1
1
1
1
For  13 : 
0
1
0
0
For  14 : 
0
1
1
1
For  15 : 
1
1
0
1
For  16 : 
0
0
0
0
For  17 : 
1
1
0
1
For  18 : 
1
1
0
0
For  19 : 
1
2
2
2
For  20 : 
1
1
1
2
For  21 : 
0
2
0
1
For  22 : 
1
2
2
2
For  23 : 
2
2
1
2
For  24 : 
1
1
1
2
For  25 : 
1
2
0
2
For  26 : 
0
1
2
2
For  27 : 
2
2
1
1
For  28 : 
1
1
0
0
For  29 : 
1
2
0
1
For  30 : 
2
2
1
2
For  31 : 
2
2
2
2
For  32 : 
0
2
1
1
For  33 : 
0
1
0
0
For  34 : 
1
2
1
2
For  35 : 
2
2
0
2
For  36 : 
2
2
2
2
For  37 : 
2
2
0
0
For  38 : 
0
2
0
2
For  39 : 
2
2
2
2
For  40 : 
1
2
0
0
For  41 : 
2
2
1
2
For  42 : 
2
2
0
1
For  43 : 
0
1
0
0
For  44 : 
1
2
0
2
For  45 : 
2
2
0
1
For  46 : 
2
2
2
2
For  47 : 
0
0
1
2
For  48 : 
2
2
1
1
For  49 : 
0
1
1
2
For  50 : 
2
2
0
2
For  51 : 
2
2
0
2
For  52 : 
1
2
0
0
For

In [200]:
for i,team in enumerate(TeamList):
    if(NumberOfMatches[i] < NumberOfWins[i]):
        print("Matches: ", NumberOfMatches[i], " Wins: ", NumberOfWins[i])

In [201]:
len(X_train)

618

In [202]:
gamecount

718

In [203]:
Y_train

[1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,


In [204]:
X_train[242]

[0.6, 1.0]

In [205]:
def BaseLineAlgorithm1(X_train):
    PredictionProbability = []
    for i,probabilities in enumerate(X_train):
        HomeTeamWinsProbability = 0
        VisitTeamWinsProbability = 0
        if(probabilities[0] == probabilities[1]):
            HomeTeamWinsProbability = 0.5
        #    VisitTeamWinsProbability = 0.5
        else:
            HomeTeamWinsProbability = probabilities[0]/(probabilities[0] + probabilities[1])
        #    VisitTeamWinsProbability = probabilities[1]/(probabilities[0] + probabilities[1])
        PredictionProbability.append(HomeTeamWinsProbability)
    return(PredictionProbability)

In [206]:
Yhat = BaseLineAlgorithm1(X_train)

In [207]:
len(Yhat)

618

In [208]:
Yhat

[0.5,
 0.5,
 1.0,
 0.5,
 0.5,
 1.0,
 0.5,
 0.0,
 0.5,
 0.5,
 1.0,
 1.0,
 0.5,
 0.5,
 0.0,
 1.0,
 0.5,
 1.0,
 1.0,
 0.3333333333333333,
 0.6666666666666666,
 0.5,
 0.3333333333333333,
 0.6666666666666666,
 0.6666666666666666,
 1.0,
 0.0,
 0.5,
 1.0,
 1.0,
 0.6666666666666666,
 0.5,
 0.0,
 0.5,
 0.5,
 1.0,
 0.5,
 1.0,
 0.5,
 0.5,
 1.0,
 0.6666666666666666,
 1.0,
 0.5,
 1.0,
 1.0,
 0.5,
 0.0,
 0.5,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.3333333333333333,
 0.6666666666666666,
 0.6666666666666666,
 0.5,
 1.0,
 0.3333333333333333,
 1.0,
 0.0,
 1.0,
 1.0,
 0.5,
 0.0,
 0.5,
 0.5,
 0.5,
 0.0,
 0.5,
 1.0,
 0.5,
 0.3333333333333333,
 0.3333333333333333,
 1.0,
 0.0,
 0.6666666666666666,
 0.0,
 0.5,
 1.0,
 0.5,
 0.0,
 0.0,
 0.5,
 0.75,
 0.5,
 0.3333333333333333,
 0.75,
 0.6666666666666666,
 0.5,
 1.0,
 0.25,
 0.5,
 0.4,
 0.5,
 1.0,
 0.5,
 0.5,
 0.4,
 0.0,
 0.5,
 0.4,
 0.4,
 0.6000000000000001,
 1.0,
 0.0,
 0.0,
 0.5,
 0.6666666666666666,
 0.6000000000000001,
 0.3333333333333333,
 0.4,
 0.5,
 1.0,
 0.0,
 

In [209]:
def Accuracy(Y_train,Yhat):
    CorrectPredictions = 0
    print(len(Y_train))
    print(len(Yhat))
    for i,current in enumerate(Y_train):
        if((Y_train[i] == 0 and Yhat[i] < 0.5) or (Y_train[i] == 1 and Yhat[i] > 0.5)):
            CorrectPredictions = CorrectPredictions + 1
    return(100 * CorrectPredictions/len(Y_train), CorrectPredictions)

In [210]:
accuracy, CorrectPredictions = Accuracy(Y_train,Yhat)

618
618


In [211]:
CorrectPredictions

338

In [212]:
accuracy

54.69255663430421

In [217]:
log_loss(Y_train,Yhat)

1.8856182646738282

In [219]:
np.log(Yhat[4])

-0.69314718055994529

In [300]:
def Log_Loss(Y_train,Yhat):
    loss = 0
    for i,current in enumerate(Y_train):
        label = Y_train[i]
        output = Yhat[i]
        current_loss = 0
        if(label == 1):
            current_loss += np.log(output)
        else:
            current_loss += np.log(1 - output)
        current_loss = max(current_loss, -34.539176193625778)
        loss += current_loss
       # print("Loss: ", current_loss, " Label: ", label," output: ", output)
    return(-loss/len(Y_train))

In [301]:
Log_Loss(Y_train,Yhat)

C:\Users\Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
C:\Users\Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


1.8856182646738198

In [296]:
t1 = [1,0]
t2 = [0,1]
log_loss(t1,t2)

34.539176193625778